In [ ]:
pip install openpyxl requests beautifulsoup4

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import time

url = 'https://www.chambredesrepresentants.ma/fr/%D8%A7%D9%84%D8%A3%D8%B3%D9%80%D8%A6%D9%84%D8%A9-%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A8%D9%8A%D8%A9-%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/fr/%2525D9%252585%2525D8%2525B1%2525D8%2525A7%2525D9%252582%2525D8%2525A8%2525D8%2525A9-%2525D8%2525A7%2525D9%252584%2525D8%2525B9%2525D9%252585%2525D9%252584-%2525D8%2525A7%2525D9%252584%2525D8%2525AD%2525D9%252583%2525D9%252588%2525D9%252585%2525D9%25258A/%2525D8%2525A7%2525D9%252584%2525D8%2525A3%2525D8%2525B3%2525D9%252580%2525D8%2525A6%2525D9%252584%2525D8%2525A9-%2525D8%2525A7%2525D9%252584%2525D9%252583%2525D8%2525AA%2525D8%2525A7%2525D8%2525A8%2525D9%25258A%2525D8%2525A9/%2525D8%2525AC%2525D8%2525A7%2525D9%252587%2525D8%2525B2%2525D9%25258A%2525D8%2525A9-%2525D8%2525A7%2525D9%252584%2525D9%252585%2525D8%2525B3%2525D8%2525AA%2525D8%2525B4%2525D9%252581%2525D9%252589-%2525D8%2525A7%2525D9%252584%2525D8%2525A5%2525D9%252582%2525D9%252584%2525D9%25258A%2525D9%252585%2525D9%25258A-%2525D9%252584%2525D9%252584%2525D8%2525AF%2525D8%2525B1%2525D9%25258A%2525D9%252588%2525D8%2525B4-%2525D9%252584%2525D8%2525AA%2525D9%252582%2525D8%2525AF%2525D9%25258A%2525D9%252585-%2525D8%2525AE%2525D8%2525AF%2525D9%252585%2525D8%2525A7%2525D8%2525AA%2525D9%252587-%2525D9%252584%2525D9%252584%2525D8%2525B3%2525D8%2525A7%2525D9%252583%2525D9%252586%2525D8%2525A9'
data = []
try:
    while url:
        response = None
        while response is None:
            try:
                response = requests.get(url)
            except requests.exceptions.RequestException as e:
                print(f"Erreur de connexion : {e}")
                time.sleep(5)  # Attendre 5 secondes avant de réessayer
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for x in soup.find_all(class_="q-b3-item q-b3i-red"):
                minister_txt = x.select_one('span:-soup-contains(" Le ministère compétent   ")')
                if minister_txt:
                    minister_txt = minister_txt.parent.text.replace(" Le ministère compétent   ", "").strip()
                else:
                    minister_txt = "none"

                question_link = x.find('a', href=True)
                if question_link:
                    question_link = question_link['href']
                    Url2 = f"https://www.chambredesrepresentants.ma{question_link}"
                    response2 = requests.get(Url2)
                    if response2.status_code == 200:
                        soup2 = BeautifulSoup(response2.text, 'html.parser')
                        
                        qnum_txt = soup2.select_one('span:-soup-contains("رقم السؤال :")')
                        if qnum_txt:
                            qnum_txt = qnum_txt.parent.text.replace("رقم السؤال :", "").strip()
                        else:
                            qnum_txt = "none"
                        
                        obj = soup2.select_one('span:-soup-contains("Objet :")')
                        if obj:
                            obj = obj.parent.text.replace("Objet :", "").strip()
                        else:
                            obj = "none"
                        
                        response_date_txt = soup2.select_one('span:-soup-contains("Date réponse :")')
                        if response_date_txt:
                            response_date_txt = response_date_txt.parent.text.replace("Date réponse :", "").strip()
                        else:
                            response_date_txt = "none"
                        
                        date_txt = soup2.select_one('span:-soup-contains("Date de la question : ")')
                        if date_txt:
                            date_txt = date_txt.parent.text.replace("Date de la question :", "").strip()
                        else:
                            date_txt = "none"
                        
                        question_txt = soup2.select_one('span:-soup-contains("Question : ")')
                        if question_txt:
                            question_txt = question_txt.parent.text.replace("Question :", "").strip()
                        else:
                            question_txt = "none"
                        
                        nom_questionneur = soup2.select_one('.q-name a')
                        if nom_questionneur:
                            nom_questionneur = nom_questionneur.text.strip()
                        else:
                            nom_questionneur = "none"
                        
                        link_questioner = soup2.select_one('.q-name').a['href']
                        Url3 = f"https://www.chambredesrepresentants.ma{link_questioner}"
                        response3 = requests.get(Url3)
                        if response3.status_code == 200:
                            soup3 = BeautifulSoup(response3.text, 'html.parser')
                            div = soup3.find('div', class_="mhr-b1-info-l").find_next('div')
                            div2 = div.find_next('div')
                            groupe = div2.find('span', class_="mht-t").find_next('span')
                            if groupe:
                                groupe = groupe.text.strip()
                            else:
                                groupe = "none"
                        
                        # Créer un dictionnaire pour les données extraites
                        entry = {
                            "minister_txt": minister_txt,
                            "qnum_txt": qnum_txt,
                            "obj": obj,
                            "response_date_txt": response_date_txt,
                            "date_txt": date_txt,
                            "question_txt": question_txt,
                            "nom_questionneur": nom_questionneur,
                            "groupe": groupe
                        }
                        
                        data.append(entry)
                        print(data)
            
            # Find the <ul> element with class "pagination"
            pagination_ul = soup.find('ul', class_='pagination')
            # Check if there is a "next" link
            next_link = pagination_ul.find('li', class_='next')
            if next_link:
                link = next_link.find('a')
                if link:
                    url1 = link['href']
                    url = f"https://www.chambredesrepresentants.ma{url1}"
                    print(f"Next Link Href: {url}")
            else:
                url = None
                        
except Exception as e:
    print(f"Une exception s'est produite : {e}")

finally:
    # Enregistrer les résultats dans un fichier JSON
    with open('resultats_ecrit.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)


[{'minister_txt': 'none', 'qnum_txt': '146', 'obj': 'موضوع ارتفاع أسعار المواد الغذائية في الآونة الأخيرة', 'response_date_txt': 'Lundi 19 Juin 2017', 'date_txt': 'Vendredi 5 Novembre 2021', 'question_txt': 'لقد أثارت الزيادات الأخيرة في أسعار المواد الغذائية سخط عدد من المواطنين الذين عبروا عن استيائهم من ارتفاع الأسعار، في ظل ما تعيشه الشرائح محدودة الدخل من أوضاع اجتماعية صعبة، وهمت هذه الزيادات سبعة مواد أساسيه دفعة واحدة، ويتعلق بالزيادة ما بين 25 و50 في المائة، كما هو الحال بالنسبة لـ”السميد”، التي ارتفع سعرها بثلاثة دراهم ونصف، منتقلا من 7 دراهم إلى 10.5 دراهم للكيلوغرام.\nوعرفت أسعار المعجنات بدورها زيادة كبيرة، تراوحت ما بين 1.70 درهما ودرهمين للكيلوغرام الواحد، بينما ارتفع سعر العدس بدوره بثلاثة دراهم ونصف، منتقلا من 12 درهما إلى 15 درهما ونصف، وارتفع سعر الكيلوغرام من الشاي بنسبة 7 في المائة\nوبخصوص مادة الزيت فقد عرف ارتفاعا صاروخيا، إذ انتقل من 10 دراهم للتر الواحد إلى 16 درهما مع إمكانية زيادة 50 سنتيما كل أسبوع. وقد حمل المواطنات والمواطنين مسؤولية هذه الزيادة التي مست ب

In [1]:
print('test')

test
